# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.
** Nota Mariana: Default hace referencia a cualquier incumplimiento de contrato por parte del deudor. **


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [2]:
import pandas as pd
import numpy as np
import json

# Para poder ver completas las descripciones de algunos dataframes 
# y series aumento la cantidad de filas y columnas que se muestran
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 152)

In [3]:
# Dado que la terminación del archivo indicaba que el csv venía comprimido entonces,
# primero lo extraje y después me encargué de leerlo
loans = pd.read_csv('LoansData_sample.csv', low_memory=False)
loans

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,NaN,15000.0,15000.0,15000.0,60 months,12.39,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000.0,Source Verified,Dec-2014,Fully Paid,n,NaN,debt_consolidation,Debt consolidation,235xx,VA,12.03,0.0,Aug-1994,750.0,754.0,0.0,NaN,NaN,6.0,0.0,138008.0,29.0,17.0,w,0.00,0.00,17392.370000,17392.37,15000.00,2392.37,0.0,0.00,0.0000,Jun-2016,12017.81,NaN,Nov-2017,684.0,680.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,149140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184500.0,NaN,NaN,NaN,5.0,29828.0,9525.0,4.7,0.0,0.0,103.0,244.0,1.0,1.0,0.0,47.0,NaN,NaN,NaN,0.0,1.0,4.0,1.0,2.0,8.0,5.0,9.0,4.0,6.0,0.0,0.0,0.0,4.0,100.0,0.0,0.0,0.0,196500.0,149140.0,10000.0,12000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1,36805548,NaN,10400.0,10400.0,10400.0,36 months,6.99,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000.0,Not Verified,Dec-2014,Charged Off,n,NaN,credit_card,Credit card refinancing,937xx,CA,14.92,0.0,Sep-1989,710.0,714.0,2.0,42.0,NaN,17.0,0.0,6133.0,31.6,36.0,w,0.00,0.00,6611.690000,6611.69,5217.75,872.67,0.0,521.27,93.8286,Aug-2016,321.08,NaN,Feb-2017,564.0,560.0,0.0,59.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,162110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19400.0,NaN,NaN,NaN,7.0,9536.0,7599.0,41.5,0.0,0.0,76.0,290.0,1.0,1.0,1.0,5.0,42.0,1.0,42.0,4.0,6.0,9.0,7.0,18.0,2.0,14.0,32.0,9.0,17.0,0.0,0.0,0.0,4.0,83.3,14.3,0.0,0.0,179407.0,15030.0,13000.0,11325.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,

In [4]:
# Para revisar la cantidad de missing values que hay en cada columna
suma = loans.isna().sum()
suma
# Y luego, comparar con el total de datos que tenemos
# loans.shape

Unnamed: 0                                         0
id                                                 0
member_id                                     100000
loan_amnt                                          0
funded_amnt                                        0
funded_amnt_inv                                    0
term                                               0
int_rate                                           0
installment                                        0
grade                                              0
sub_grade                                          0
emp_title                                       5264
emp_length                                      5259
home_ownership                                     0
annual_inc                                         0
verification_status                                0
issue_d                                            0
loan_status                                        0
pymnt_plan                                    

In [5]:
# Revisando tipos de datos en algunas columnas
# col = loans['settlement_term']
# col1 = col.dropna()
# col1

## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [6]:
# Es una serie: key - nombre de la columna, value - tipo de dato
column_types = loans.dtypes
column_types

# Parece ser que no hay columnas duplicadas, ni columnas vacias.
# columns = loans.columns
# columns

Unnamed: 0                                      int64
id                                              int64
member_id                                     float64
loan_amnt                                     float64
funded_amnt                                   float64
funded_amnt_inv                               float64
term                                           object
int_rate                                      float64
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
verification_status                            object
issue_d                                        object
loan_status                                    object
pymnt_plan                  

## Cargar descripción de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [7]:
datos_dict = pd.read_excel('https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']

**Nota Mariana**: me arrojaba un error de lectura porque no tenia instalada openpyxl. 
Tuve que meterme a terminal y correr el comando 'pip install openpyxl' 

In [8]:
# Revisando la descripción
datos_dict

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


In [9]:
# Dataframe que no contiene las columnas vacias
df_info = datos_dict.dropna(subset='feature')
df_info

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
5,annual_inc_joint,The combined self-reported annual income provi...
6,application_type,Indicates whether the loan is an individual ap...
7,avg_cur_bal,Average current balance of all accounts
8,bc_open_to_buy,Total open to buy on revolving bankcards.
9,bc_util,Ratio of total current balance to high credit/...


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [10]:
# Codigo para guardar el df en formato pickle
df_info.to_pickle("df_info.pkl")

In [11]:
# Codigo para cargar del pickle al df
df_lec = pd.read_pickle("df_info.pkl")
df_lec

,feature,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
5,annual_inc_joint,The combined self-reported annual income provi...
6,application_type,Indicates whether the loan is an individual ap...
7,avg_cur_bal,Average current balance of all accounts
8,bc_open_to_buy,Total open to buy on revolving bankcards.
9,bc_util,Ratio of total current balance to high credit/...


## EDA

## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [12]:
print("Original_dtypes:")
print(loans.dtypes)

Original_dtypes:
Unnamed: 0                                      int64
id                                              int64
member_id                                     float64
loan_amnt                                     float64
funded_amnt                                   float64
funded_amnt_inv                               float64
term                                           object
int_rate                                      float64
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
verification_status                            object
issue_d                                        object
loan_status                                    object
pymnt_plan 

El método `convert_dtypes()` nos permite castear todo el dataframe (por columna) para que se obtenga el mejor tipo de dato posible. Después de revisar con cuidado los tipos de datos que utiliza la función, éstos me parecieron adecuados porque me facilitarían la imputación de Nans.

Link de la documentación de `convert_dtypes()`: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.convert_dtypes.html

In [13]:
new_loans = loans.convert_dtypes()
print("Nuevos tipos:")
print(new_loans.dtypes)

Nuevos tipos:
Unnamed: 0                                      Int64
id                                              Int64
member_id                                       Int64
loan_amnt                                       Int64
funded_amnt                                     Int64
funded_amnt_inv                                 Int64
term                                           string
int_rate                                      Float64
installment                                   Float64
grade                                          string
sub_grade                                      string
emp_title                                      string
emp_length                                     string
home_ownership                                 string
annual_inc                                    Float64
verification_status                            string
issue_d                                        string
loan_status                                    string
pymnt_plan    

In [14]:
# Viendo si el metodo convert_dtypes() sirve para mantener los valores en NA
# Resultado: funciona adecuadamente

# prueba = new_loans['member_id']
# x = prueba.dropna()
# x

## Borrando columnas inservibles

In [15]:
# Revisando la cantidad de Nans por columna.
suma2 = new_loans.isna().sum()
suma2

Unnamed: 0                                         0
id                                                 0
member_id                                     100000
loan_amnt                                          0
funded_amnt                                        0
funded_amnt_inv                                    0
term                                               0
int_rate                                           0
installment                                        0
grade                                              0
sub_grade                                          0
emp_title                                       5264
emp_length                                      5259
home_ownership                                     0
annual_inc                                         0
verification_status                                0
issue_d                                            0
loan_status                                        0
pymnt_plan                                    

**¿Por qué decidí borrar pocas columnas?**

Cuando estamos tratando con datos respectivos a préstamos y deuda es importante mantener la mayor cantidad posible de datos del deudor.

No queremos que por una falta de datos se llegue a tomar una decisión inadecuada, resultando en `pérdida de dinero`. Ya sea por brindar préstamos a clientes que no cumplen los requisitos y que son probables a cometer 'default'. O al contrario, evitar que se brinden préstamos a los clientes que sí cumplen los requisitos, pero que por falta de información no se haga.

Además, otra consideración que se tuvo fue la integridad de los datos. Todas las columnas que fueron eliminadas eran columnas `totalmente vacías` y que, de ser necesario, `pueden ser reconstruidas` con datos de otras columnas. Por ejemplo, la columna 'open_rv_12m' puede reconstruirse a partir de 'open_rv_24m'.

In [16]:
# Columnas por borrar
lista_borrar = ['member_id', 'desc', 'annual_inc_joint', 'dti_joint', 'open_acc_6m', 'total_cu_tl','revol_bal_joint','open_il_12m', 'open_rv_12m']
final_loans = new_loans.drop(lista_borrar, axis=1)
final_loans.shape
final_loans

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,15000,15000,15000,60 months,12.39,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000.0,Source Verified,Dec-2014,Fully Paid,n,debt_consolidation,Debt consolidation,235xx,VA,12.03,0,Aug-1994,750,754,0,<NA>,<NA>,6,0,138008,29.0,17,w,0.0,0.0,17392.37,17392.37,15000.0,2392.37,0.0,0.0,0.0,Jun-2016,12017.81,<NA>,Nov-2017,684,680,0,<NA>,1,Individual,<NA>,0,0,149140,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,184500,<NA>,<NA>,5,29828,9525,4.7,0,0,103,244,1,1,0,47,<NA>,<NA>,<NA>,0,1,4,1,2,8,5,9,4,6,0,0,0,4,100.0,0.0,0,0,196500,149140,10000,12000,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Cash,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,1,36805548,10400,10400,10400,36 months,6.99,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000.0,Not Verified,Dec-2014,Charged Off,n,credit_card,Credit card refinancing,937xx,CA,14.92,0,Sep-1989,710,714,2,42,<NA>,17,0,6133,31.6,36,w,0.0,0.0,6611.69,6611.69,5217.75,872.67,0.0,521.27,93.8286,Aug-2016,321.08,<NA>,Feb-2017,564,560,0,59,1,Individual,<NA>,0,0,162110,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,19400,<NA>,<NA>,7,9536,7599,41.5,0,0,76,290,1,1,1,5,42,1,42,4,6,9,7,18,2,14,32,9,17,0,0,0,4,83.3,14.3,0,0,179407,15030,13000,11325,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Cash,N,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2,37842129,21425,21425,21425,60 months,15.59,516.36,D,D1,Programming Analysis Supervisor,6 years,RENT,63800.0,Source Verified,Dec-2014,Fully Paid,n,credit_card,Credit card refinancing,658xx,MO,18.49,0,Aug-2003,685,689,0,60,<NA>,10,0,16374,76

## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Imputación

In [17]:
final_loans['emp_title'] = final_loans['emp_title'].fillna('Not supplied')

# Verificando si se hizo la modificacion de valores adecuadamente.
# arr = final_loans['emp_title'] == 'Not supplied'
# arr.value_counts()

lista1 = ('open_act_il', 
          'open_il_24m', 
          'total_bal_il', 
          'open_rv_24m', 
          'max_bal_bc',
          'inq_fi',
          'inq_last_12m',
          'bc_open_to_buy',
          'num_tl_120dpd_2m',
          'sec_app_earliest_cr_line',
          'sec_app_inq_last_6mths',
          'sec_app_mort_acc',
          'sec_app_open_acc',
          'sec_app_open_act_il',
          'sec_app_num_rev_accts',
          'sec_app_chargeoff_within_12_mths',
          'sec_app_collections_12_mths_ex_med',
          'deferral_term',
          'hardship_amount',
          'hardship_length' ,
          'hardship_dpd',
          'orig_projected_additional_accrued_interest',
          'hardship_payoff_balance_amount',
          'hardship_last_payment_amount',
          'settlement_amount',
          'settlement_percentage',
          'settlement_term',
          'verification_status_joint')

for i in range(len(lista1)):
    val = lista1[i]
    # print(val)
    final_loans[val] = final_loans[val].fillna(0)
    # print(final_loans[val].value_counts())


lista2 = ('last_pymnt_d',
          'next_pymnt_d',
          'last_credit_pull_d', 
          'hardship_type',
          'hardship_reason',
          'hardship_status',
          'hardship_start_date', 
          'hardship_end_date',
          'payment_plan_start_date',
          'hardship_loan_status',
          'debt_settlement_flag_date',
          'settlement_status',
          'settlement_date')

for i in range(len(lista2)):
    val2 = lista2[i]
    # print(val2)
    final_loans[val2] = final_loans[val2].fillna('Not applicable')
    # print(final_loans[val2].value_counts())

final_loans


,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,0,38098114,15000,15000,15000,60 months,12.39,336.64,C,C1,MANAGEMENT,10+ years,RENT,78000.0,Source Verified,Dec-2014,Fully Paid,n,debt_consolidation,Debt consolidation,235xx,VA,12.03,0,Aug-1994,750,754,0,<NA>,<NA>,6,0,138008,29.0,17,w,0.0,0.0,17392.37,17392.37,15000.0,2392.37,0.0,0.0,0.0,Jun-2016,12017.81,Not applicable,Nov-2017,684,680,0,<NA>,1,Individual,0,0,0,149140,0,0,<NA>,0,<NA>,0,0,<NA>,184500,0,0,5,29828,9525,4.7,0,0,103,244,1,1,0,47,<NA>,<NA>,<NA>,0,1,4,1,2,8,5,9,4,6,0,0,0,4,100.0,0.0,0,0,196500,149140,10000,12000,<NA>,<NA>,0,0,0,0,<NA>,0,0,0,0,<NA>,N,Not applicable,Not applicable,Not applicable,0,0.0,Not applicable,Not applicable,Not applicable,0,0,Not applicable,0.0,0.0,0.0,Cash,N,Not applicable,Not applicable,Not applicable,0.0,0.0,0
1,1,36805548,10400,10400,10400,36 months,6.99,321.08,A,A3,Truck Driver Delivery Personel,8 years,MORTGAGE,58000.0,Not Verified,Dec-2014,Charged Off,n,credit_card,Credit card refinancing,937xx,CA,14.92,0,Sep-1989,710,714,2,42,<NA>,17,0,6133,31.6,36,w,0.0,0.0,6611.69,6611.69,5217.75,872.67,0.0,521.27,93.8286,Aug-2016,321.08,Not applicable,Feb-2017,564,560,0,59,1,Individual,0,0,0,162110,0,0,<NA>,0,<NA>,0,0,<NA>,19400,0,0,7,9536,7599,41.5,0,0,76,290,1,1,1,5,42,1,42,4,6,9,7,18,2,14,32,9,17,0,0,0,4,83.3,14.3,0,0,179407,15030,13000,11325,<NA>,<NA>,0,0,0,0,<NA>,0,0,0,0,<NA>,N,Not applicable,Not applicable,Not applicable,0,0.0,Not applicable,Not applicable,Not applicable,0,0,Not applicable,0.0,0.0,0.0,Cash,N,Not applicable,Not applicable,Not applicable,0.0,0.0,0
2,2,37842129,21425,21425,21425,60 months,15.59,516.36,D,D1,Programming Analysis Supervisor,6 years,RENT,63800.0,Source Verified,Dec-2014,Fully Paid,n,credi

In [18]:
# suma3 = final_loans.isna().sum()
# suma3

### Código para salvar y cargar JSONs

In [38]:
# Para obtener la lista de columnas a las que no se les imputaron valores.
aux = datos_dict['feature'].dropna()
listaux = list(aux)
lista3 = [elem for elem in listaux if ((elem not in lista1) and (elem not in lista2) and (elem not in lista_borrar))]
#lista3


In [63]:
# Formato del json {'nombre de la columna' : {'estrategia' : 'identificador', 'valor' : 'missing'}}
dic = dict()
for col in lista1:
    dic[col] = {'estrategia' : 'cero', 'valor' : 0}

for col in lista2:
    dic[col] = {'estrategia' : 'identificador', 'valor' : 'Not applicable'}

for col in lista3:
    dic[col] = {'estrategia' : 'ninguna', 'valor' : None}

dic['emp_title'] = {'estrategia' : 'identificador', 'valor' : 'Not supplied'}

dic

{'open_act_il': {'estrategia': 'cero', 'valor': 0},
 'open_il_24m': {'estrategia': 'cero', 'valor': 0},
 'total_bal_il': {'estrategia': 'cero', 'valor': 0},
 'open_rv_24m': {'estrategia': 'cero', 'valor': 0},
 'max_bal_bc': {'estrategia': 'cero', 'valor': 0},
 'inq_fi': {'estrategia': 'cero', 'valor': 0},
 'inq_last_12m': {'estrategia': 'cero', 'valor': 0},
 'bc_open_to_buy': {'estrategia': 'cero', 'valor': 0},
 'num_tl_120dpd_2m': {'estrategia': 'cero', 'valor': 0},
 'sec_app_earliest_cr_line': {'estrategia': 'cero', 'valor': 0},
 'sec_app_inq_last_6mths': {'estrategia': 'cero', 'valor': 0},
 'sec_app_mort_acc': {'estrategia': 'cero', 'valor': 0},
 'sec_app_open_acc': {'estrategia': 'cero', 'valor': 0},
 'sec_app_open_act_il': {'estrategia': 'cero', 'valor': 0},
 'sec_app_num_rev_accts': {'estrategia': 'cero', 'valor': 0},
 'sec_app_chargeoff_within_12_mths': {'estrategia': 'cero', 'valor': 0},
 'sec_app_collections_12_mths_ex_med': {'estrategia': 'cero', 'valor': 0},
 'deferral_term'

In [66]:
# Codigo para salvar el json file
with open('estrategias.json', 'w') as js:
    json.dump(dic, js)

In [67]:
# Codigo para cargar el json
with open('estrategias.json', 'r') as json_file:
    prueba_dic = json.load(json_file)

{'open_act_il': {'estrategia': 'cero', 'valor': 0},
 'open_il_24m': {'estrategia': 'cero', 'valor': 0},
 'total_bal_il': {'estrategia': 'cero', 'valor': 0},
 'open_rv_24m': {'estrategia': 'cero', 'valor': 0},
 'max_bal_bc': {'estrategia': 'cero', 'valor': 0},
 'inq_fi': {'estrategia': 'cero', 'valor': 0},
 'inq_last_12m': {'estrategia': 'cero', 'valor': 0},
 'bc_open_to_buy': {'estrategia': 'cero', 'valor': 0},
 'num_tl_120dpd_2m': {'estrategia': 'cero', 'valor': 0},
 'sec_app_earliest_cr_line': {'estrategia': 'cero', 'valor': 0},
 'sec_app_inq_last_6mths': {'estrategia': 'cero', 'valor': 0},
 'sec_app_mort_acc': {'estrategia': 'cero', 'valor': 0},
 'sec_app_open_acc': {'estrategia': 'cero', 'valor': 0},
 'sec_app_open_act_il': {'estrategia': 'cero', 'valor': 0},
 'sec_app_num_rev_accts': {'estrategia': 'cero', 'valor': 0},
 'sec_app_chargeoff_within_12_mths': {'estrategia': 'cero', 'valor': 0},
 'sec_app_collections_12_mths_ex_med': {'estrategia': 'cero', 'valor': 0},
 'deferral_term'

In [69]:
# Revision final del df
# final_loans